### 4.2 손실 함수

신경망 학습은 현재 상태를 하나의 지표로 표현한다. 그 지표를 가장 좋게 만드는 가중치 매개변수의 값을 탐색하는 것이다.  
신경망에서의 지표는 바로 손실 함수이다. loss function.  

손실 함수는 신경망 성능의 나쁨을 나타내는 지표이다. 얼마나 훈련 데이터를 잘 처리하지 못하는지를 나타낸다.  
여기서 마이너스만 곱하면 얼마나 좋은지에 대한 지표로 변한다.  
나쁨과 좋음 어느 쪽을 지표로 삼아도 본질적으로 수행하는 일은 같다.  


#### 4.2.1 오차제곱합

가장 많이 쓰이는 손실 함수는 오차제곱합(sum of squares for error, SSE)이다.  
신경망의 출력값 yk와 정답 레이블의 tk, 데이터의 차원 수를 나타내는 k로 구성되는 식이다.  
소프트맥스 함수의 출력 배열과 원 핫 인코딩의 정답 레이블의 차이를 제곱한 뒤 그 총합을 구한다.  

In [8]:
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t) ** 2)
                        
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 정답이 2일때
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # 추론 값이 2가 제일 높을 때
print(sum_squares_error(np.array(y), np.array(t))) # 오차가 더 작다

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] # 추론 값이 7이 제일 높을 때
print(sum_squares_error(np.array(y), np.array(t))) # 오차가 크다

0.09750000000000003
0.5975


#### 4.2.2 교차 엔트로피 오차

Cross Entropy Error (CSEE)도 자주 이용한다.  
신경망의 출력 yk에 밑이 e인 자연로그를 취해서 실질적으로 정답일때의 추정의 자연로그를 계산하는 식이 된다.  
답이 아닌 모두는 0이니 logyk와 곱해도 0이 되서 영향을 주지 않는다.  
엔트로피 오차는 정답일 때의 출력이 전체 값을 정하게 된다.  

x가 1이 될때 y는 0이 되고, x가 0에 가까워 질 수록 y는 점점 작아진다.  
정답에 해당하는 출력이 커질수록 0에 다가가다가, 그 출력이 1일때 0이 된다.   
정답일때의 출력이 작아질 수록 오차는 커지게 된다.  

In [10]:
def cross_entrophy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

    # 아주 작은 값인 delta 값을 더하는 이유는 np.log에 0을 입력하면 마이너스 무한대를 뜻하는 -inf가 떠서 계산이 안되기 때문이다.
    
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 정답이 2일때
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # 추론 값이 2가 제일 높을 때
print(cross_entrophy_error(np.array(y), np.array(t))) # 오차가 더 작다

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] # 추론 값이 7이 제일 높을 때
print(cross_entrophy_error(np.array(y), np.array(t))) # 오차가 크다

0.510825457099338
2.302584092994546


#### 4.2.3 미니배치 학습

기계학습은 훈련 데이터를 통해 학습한다. 정확히는 훈련데이터에 대한 손실함수 값을 구해서 그 값을 최대한 줄여주는 매개변수를 찾아낸다.  
그럴려면 모든 데이터를 대상으로 손실 함수 값을 구해야 하는데 이는 어떻게 구현할까?  
하나의 데이터에 대한 식을 N개에 대해 확장하면 된다. 다만 마지막에 N으로 나누어 평균 손실 함수를 구한다.  

근데 데이터의 양이 너무 많을 경우는 일부를 추려 전체의 근사치를 구한다.  
훈련 데이터에서 일부만 골라 학습을 수행하는 것을 미니배치 mini batch라고 한다.

In [12]:
# MNIST 데이터셋을 불러오는 함수
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \ # 훈련 데이터와 시험 데이터를 원-핫 인코딩으로 불러온다.
    load_mnist(normalize = True, one_hot_label = True)

print(x_train.shape, t_train.shape)

(60000, 784) (60000, 10)


In [15]:
# 무작위로 10장만 빼내는 코드
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

print(batch_mask, x_batch, t_batch)

# np.random.choice() 예제
print(np.random.choice(60000, 10))

[ 1393 42487  3110 20910 55076 28127 40389 12647 34485 12392] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
[40800 58695 38352 55724 21589 49227 40305  7737 30981 53636]


#### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

미니배치를 지원하는 교차 엔트로피 오차를 구현해보자!  
방금 구현한 데이터를 하나씩 처리하는 부분을 수정하면 된다.  
1개나 여러개의 경우 모두 처리할 수 있도록 하자.  

In [16]:
# 라벨이 원-핫 인코딩인 경우
def cross_entrophy_error(y, t):
    if y.ndim == 1:        # 만약 y가 1차원이라면 데이터가 하나라는 뜻이므로 데이터의 형상을 바꿔준다.
        t.reshape(1, t.size)
        y.reshape(1, y.size)
    
    batch_size = y.shape[0] #배치의 크기로 나누어 정규화를 한다.
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [ ]:
# 라벨이 숫자인 경우
def cross_entrophy_error(y, t):
    if y.ndim == 1:        # 만약 y가 1차원이라면 데이터가 하나라는 뜻이므로 데이터의 형상을 바꿔준다.
        t.reshape(1, t.size)
        y.reshape(1, y.size)
    
    batch_size = y.shape[0] #배치의 크기로 나누어 정규화를 한다.
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

원-핫 인코딩의 경우 어차피 정답을 제외한 라벨은 모두 0이기에 교차 엔트로피 오차도 0이다.  
그래서 그 계산은 무시하고 신경망의 출력만으로 계산한다.

y에는 네트워크가 출력한 예측값들이 batch_szie만큼 들어있을건데, np.arange로 batch_size만큼 인덱스를 매겨 해당 인덱스와 원래 라벨 값이랑 쌍으로 묶어서 출력한다.  
y[np.arange(batch_size), t]은 각 데이터의 정답 레이블에 해당하는 신경망의 출력을 추출한다.  

#### 4.2.5 왜 손실 함수를 설정하는가?

높은 정확도의 매개변수를 찾는 것이라면 왜 굳이 손실 함수의 값이라는 우회적인 방법을 택할까?  
신경망 학습에서의 미분 역할에 주목하면 된다.  

신경망 학습에서는 최적의 매개변수를 탐색할 때 손실 함수의 값을 가능한 작게 하는 매개변수 값을 찾는데, 이때 매개변수의 미분 (기울기)를 계산해서 서서히 갱신한다.  

가상의 신경망이 있고 어느 한 가중치 매개변수에 주목한다고 해보자. 그 가중치 매개변수의 손실 함수의 미분이란, 가중치 매개변수의 값을 아주 조금 변화 시켰을때, 손실 함수가 어떻게 변하나 라는 의미이다.  
만약 이 미분 값이 음수면 그 가중치 매개변수를 양의 방향으로 변화시켜 손실함수의 값을 줄이고, 반대라면 음의 방향으로 변화시켜 값을 줄일 수 있다.  

정확도를 지표로 삼아서는 안되는 이유는 미분 값 대부분의 장소에서 0이되서 매개변수 갱신이 불가능하기 때문이다.  
왜 정확도의 미분 값은 대부분의 장소에서 0이 되는걸까?


정확도를 지표로 삼으면 100장중 32장을 맞게 된 경우는 32%가 된다. 근데 이는 매개변수를 조정해도 정확도가 변하지 않는다. 매개변수의 변화와 상관없이 32장 맞으면 32%인거고, 33장을 맞으면 33%인것이다. 정확도가 계산되도 연속적이 아닌 것이다.  
반면 손실함수는 매개변수의 값이 조금 바뀔때마다 손실함수의 값도 연속적으로 바뀌게 된다.  
이는 계단 함수를 활성화 함수로 사용하지 않는 이유와도 들어맞는다. 계단 함수의 미분은 0이외의 곳에서 0이 되어 손실함수를 지표로 삼는게 아무런 의미가 없다. 매개변수의 작은 변화가 주는 파장을 계단함수가 말살해서 손실 함수의 값에는 변화가 없기 때문이다.  
다만 시그모이드는 출력이 연속적으로 변하며 곡선의 기울기도 연속적으로 변한다. 기울기가 0이 되지 않기에 올바른 학습이 가능하다.  